# DataFrame Data Type Conversion

The previous chapters in this part focused on the understanding and construction of data types outside of the context of real data. We mainly used numpy arrays and pandas Series for exploring these data types. In this chapter, we will work with entire DataFrames of real data. 

Let's see some examples by reading in a few of the columns from the college dataset. The `usecols` parameter is set to just six columns, one of which, the institution name, is placed in the index.

In [ ]:
import pandas as pd
cols = ['instnm', 'hbcu', 'relaffil', 'ugds', 
        'md_earn_wne_p10', 'grad_debt_mdn_supp']
college = pd.read_csv('../data/college.csv', index_col='instnm', usecols=cols)
college.head(3)

From the above visual display, it appears that all of the columns are numeric. But, unlike Series, we cannot tell with DataFrames, as each column's specific data type is not displayed in the output. You must access them with the `dtypes` attribute. 

In [ ]:
college.dtypes

You might be surprised to see that the last two columns are read in as objects. They certainly appear to be numeric. If you look closely at the values in the `grad_debt_mdn_supp` column, you'll notice that the second value has a single decimal value while the others do not. Columns that have the float data type will never have a visual display like this. The decimal places will always align.

When a seemingly numeric column is read in as an object, it is a clue that there are strings in this column. One of the first things we can do to investigate this issue is output some of the values from the underlying numpy array with the `values` attribute. Here, we take a look at the first five values to see that they strings.

In [ ]:
college['grad_debt_mdn_supp'].head().values

If the `read_csv` function initially reads in a column with the data type object, then you can guarantee that all non-missing values will be strings. We can get the exact type of an individual value by extracting it from the DataFrame and passing it to the built-in `type` function.

In [ ]:
val = college.loc['Alabama A & M University', 'grad_debt_mdn_supp']
val

In [ ]:
type(val)

The `read_csv` function won't read in a column of data as strings unless it contains non-numeric characters. One way to find non-numeric characters is to sort the string column in descending order. Numeric characters have a lower unicode code point than alphabetic characters, so this should put the alphabetic strings to the top. 

Instead of sorting the column itself, we first call the `value_counts` method and then sort the index in descending order to place all the non-numeric values at the very top. This also reveals the frequency of occurrence.

In [ ]:
(college['grad_debt_mdn_supp'].value_counts()
                              .sort_index(ascending=False).head(3))

This method isn't perfect for uncovering non-numeric strings since it's possible a string can begin with a digit only to be followed by alphabetic characters. Regular expressions (which are covered in an upcoming part) are needed to search for more specific patterns.

### Converting non-numeric values to missing

We've uncovered one string, `'PrivacySuppressed'`, in the `grad_debt_mdn_supp` column which forced pandas to use object instead of float for its data type. If we want to use this column as a float, we'll need to convert the strings to some numeric value or a missing value.

Let's go ahead and convert the string to a missing value. As we saw previously, we cannot use the `astype` method to make this conversion as the `errors` parameter cannot be set to 'coerce'. We need to use the `to_numeric` function to convert this column to a float. We set the `errors` parameter to 'coerce' to force any value that isn't able to be converted to missing. One minor annoyance is that `to_numeric` converts only a single column at a time. Below, we overwrite both of the object columns (the other column has the same exact issue) with two separate calls of the `to_numeric` function.

In [ ]:
college['grad_debt_mdn_supp'] = pd.to_numeric(college['grad_debt_mdn_supp'], 
                                              errors='coerce')
college['md_earn_wne_p10'] = pd.to_numeric(college['md_earn_wne_p10'], 
                                           errors='coerce')
college.head(3)

Notice how the decimals align. Let's verify that the new data types are float.

In [ ]:
college.dtypes

## The `astype` method for DataFrames

The `astype` method is still useful for DataFrames. We can convert all columns at once to a different type. Below, we convert each column to a 32-bit float and immediately verify the data types.

In [ ]:
college.astype('float32').dtypes

You can change the data type of specific columns by using a dictionary to map the column name to the desired type. Here, we change `relaffil` to an 8-bit integer and `ugds` to a 32-bit float.

In [ ]:
college.astype({'relaffil': 'int8', 'ugds': 'float32'}).dtypes

## Reading in data with known missing values

You can avoid having to use `to_numeric` if you know the missing value representation in your dataset before you read in your data. Set the `na_values` parameter of the `read_csv` function to the string that represents missing values. You can use a list to specify more values and a dictionary to specify different missing values for each column. Here, we read in our college dataset again and convert every occurrence of 'PrivacySuppressed' to missing on read. You can also use the `dtype` parameter to set the data type of specific columns on read with a dictionary.

In [ ]:
college = pd.read_csv('../data/college.csv', index_col='instnm', 
                      usecols=cols, na_values='PrivacySuppressed', 
                      dtype={'relaffil': 'int8', 'ugds': 'float32'})
college.head(3)

It appears as though the last two columns were read in as floats. Let's verify that pandas has correctly read in the last two columns as floats.

In [ ]:
college.dtypes

## More data type conversion with the housing dataset

Let's see another example of converting data types on DataFrames with the housing dataset. We begin by reading in just seven of the columns by setting the `usecols` parameter to a list of those column names.

In [ ]:
cols = ['MSSubClass', 'LotConfig', 'Condition1', 'YearBuilt', 
        'ExterQual', 'OverallQual', 'SalePrice']
housing = pd.read_csv('../data/housing.csv', usecols=cols)
housing.head()

Let's see how pandas has chosen to read in this data.

In [ ]:
housing.dtypes

By examining the data dictionary, 'MSSubClass', 'LotConfig', and 'Condition1' are nominal categorical data. Both 'OverallQual' and 'ExterQual' are ordinal, and 'YearBuilt' and 'SalePrice' are integers. A dictionary mapping each column to its new data type is created.

Ordered categorical data types must be constructed with the `CategoricalDtype` constructor. We also choose unsigned integers with less bit sizes for the 'YearBuilt' and 'SalePrice' columns. Finally, we pass this dictionary to the `astype` method to make the conversion. It is not necessary to specify every column like we did here. Those not specified retain their data type.

In [ ]:
oq_dtype = pd.CategoricalDtype(range(1, 11), ordered=True)
eq_dtype = pd.CategoricalDtype(['Po', 'Fa', 'TA', 'Gd', 'Ex'], ordered=True)
dtype_dict = {'MSSubClass': 'category', 
              'LotConfig': 'category', 
              'Condition1': 'category',
              'YearBuilt': 'uint16', 
              'SalePrice': 'uint32',
              'OverallQual': oq_dtype, 
              'ExterQual': eq_dtype}
housing2 = housing.astype(dtype_dict)
housing2.dtypes

This data type conversion may be done on read by using the same dictionary passed to the `dtype` parameter.

In [ ]:
housing2 = pd.read_csv('../data/housing.csv', usecols=cols, dtype=dtype_dict)
housing2.dtypes

The amount of memory saved by converting to these new data types is substantial with most of it coming from the change to categorical. Let's get the memory usage for both the original and converted datasets.

In [ ]:
orig_mem = housing.memory_usage(index=False, deep=True)
orig_mem

In [ ]:
new_mem = housing2.memory_usage(index=False, deep=True)
new_mem

Summing these two Series gets the total memory used for each. Dividing these totals reveals a reduction in memory of 94 percent.

In [ ]:
1 - new_mem.sum() / orig_mem.sum()

## Exercises

### Exercise 1

<span style="color:green; font-size:16px">Read in the bikes dataset and select the `tripduration` column. Find its data type and then use the `memory_usage` method to find how much memory (in bytes) it is using. Change its data type to the smallest possible type so that no information is lost. What percentage of memory has been saved?</span>

### Exercise 2

<span style="color:green; font-size:16px">Read in the diamonds dataset and convert the data types of each column so they use the least amount of memory without losing any information.</span>